<a href="https://colab.research.google.com/github/zudanievil/MiceBrainMriSegmentation/blob/paper/ipynb/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open this notebook in Google Colab"/></a>

This notebook can run in Google Colab with no prior setup.
You can also run it on a local python installation, but you will need to change some CAPITAL_CASE constants.

# Mice Brain Mri Segmentation #
(actually, it can segment human scans as well. the repo name is a bit underthought)

## Overview ##
* These scripts allow to finenly segment raster images (MRI scans) using svg images (from histological brain atlas).
* Atlas svg images and some other assets are retrieved from Allen Brain Institute web-service, which is free for academic and personal use.
    * [Allen Brain Institute website](https://portal.brain-map.org/)
    * [Mouse | Human brain atlases](http://atlas.brain-map.org/)
    * [Allen Brain Institute API help](http://help.brain-map.org/display/api/Allen+Brain+Atlas+API)
* This repository contains a collection of cohesive scripts, **not a Python package or an application**. One should know Python to use this.

---

This notebook will guide through one through the path to replicate our work:
1. [installing nesessary programs](#1.-Installations)
1. [creating folders](#2a.-creating-folders) and nessesary files
1. [manually segmenting the brains from the scans](#3.-manual-scan-segmentation)
1. [bulk-downloading atlas images](#4a.-downloading-atlas-images) and generating raster masks from them
1. [preprocessing scans before fine segmentation](#5.-preprocessing-scans)
1. [fine-segmenting brains into anatomical regions](#6.-fine-anatomical-segmentation)
1. [postprocessing segmentation results](#7.-aftermath)

## 1. Installations
This notebook can run in [`GOOGLE COLAB`](https://colab.research.google.com/github/zudanievil/MiceBrainMriSegmentation/blob/paper/ipynb/example.ipynb) with no prior setup, but it is best to run it on a locally installed python.

To do this on a local machine:
1. clone this repository
1. install [Anaconda] package manager
1. [create virtual environment from the file] named `conda_env.yml` contained in this repository
1. install [Inkscape], install [ImageJ]
1. it is best to install [PyCharm] for code editing, and [set up PyCharm to use Anaconda virtual environment]

[Anaconda]: https://www.anaconda.com/products/individual
[create virtual environment from the file]: https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file
[PyCharm]: https://www.jetbrains.com/pycharm/download/
[set up PyCharm to use Anaconda virtual environment]: https://www.jetbrains.com/help/pycharm/conda-support-creating-conda-virtual-environment.html
[Inkscape]: https://inkscape.org/
[ImageJ]: https://imagej.nih.gov/ij/download.html


### imports & utility functions

In [1]:
import sys
from pathlib import Path
from urllib.request import urlretrieve
from zipfile import ZipFile
import yaml
from shutil import copy as file_copy

In [2]:
# @markdown some generic utility functions, used throughout the notebook

def unzip(src: Path, dst: Path):
    """unzip a `.zip file located at `src` into `dst` folder"""
    with ZipFile(src, 'r') as z:
        z.extractall(dst)
    
    
def rmdir_rec(folder: Path): #why didn't they implement it in the pathlib?
    """removes `folder` with all the contents"""
    for f in folder.iterdir():
        rmdir_rec(f) if f.is_dir() else f.unlink()
    folder.rmdir()
    
    
def merge_folders(src: Path, dst: Path, matching_files_policy: {"skip", "replace", "rename", "error"} = "replace"):
    """
    recursively copies contents from the `src` folder to `dst` folder.
    if files match, uses `matching_files_policy`:
        - skip
        - replace
        - error: raises FileExistsError
        - rename: adds ".copy" suffix to filename, eg `a.txt` -> `a.copy.txt`
    """
    assert matching_files_policy in {"skip", "replace", "rename", "error"}
    
    for path_src in src.iterdir():
        path_dst = dst / path_src.name
        if path_src.is_dir():
            path_dst.mkdir(exist_ok=True)
            merge_folders(path_src, path_dst, matching_files_policy)
        else:
            if path_dst.exists():
                if matching_files_policy == "error":
                    raise FileExistsError(path_dst)
                elif matching_files_policy == "replace":
                    path_dst.unlink()
                elif matching_files_policy == "rename":
                    while path_dst.exists():
                        path_dst = path_dst.parent / (path_dst.stem + ".copy" + path_dst.suffix)
                else:
                    continue
            file_copy(path_src, path_dst)


def fetch_unpack_zip(url: str, dst: Path) -> "http message":
    """
    fetches example data from the cloud, 
    :returns http.client.HTTPMessage
    """
    zip, msg = urlretrieve(url)
    zip = Path(zip)
    unzip(src=zip, dst=dst)
    zip.unlink()
    return msg


def fetch_github_repo(
    user: str, repo: str, branch: str, 
    dst_dir: Path, compression: {"zipball", "tarball"} = "zipball",
) -> (str, "http message"):
    url = f"https://api.github.com/repos/{user}/{repo}/{compression}/{branch}" 
    # https://docs.github.com/en/rest/reference/repos#download-a-repository-archive-tar
    dst_dir = Path("/content") 
    
    zipball, msg = urlretrieve(url)
    unzip(src=Path(zipball), dst=dst_dir)
    Path(zipball).unlink()
    for p in dst_dir.iterdir():
        if p.name.startswith(f"{user}-{repo}"):
            # there is a seemingly random number at the end of the folder name
            new_p = p.parent / f"{user}-{repo}"
            p.rename(new_p)
            return str(new_p), msg


def file_backup(p: Path):
    file_copy(p, p.parent / (p.name + ".backup"))


def modify_info_folder_config(info_instance, keys: list, value: any):
    config = info_instance.configuration()
    config_path = info_instance.configuration_path()

    c = config
    for k in keys[0:-1]:
        c = c[k]
    c[keys[-1]] = value

    with config_path.open("wt") as f:
        yaml.safe_dump(config, f)

### Constants

In [3]:
RUNTIME_IS_ON_GOOGLE_COLAB: bool = "google.colab" in sys.modules


WORKING_DIR = "/content/example" if RUNTIME_IS_ON_GOOGLE_COLAB \
    else YOUR_TEMP_DIRECTORY  # if you run this locally, define your temp directory 


# if on google colab, fetch whole repository from github
if RUNTIME_IS_ON_GOOGLE_COLAB:
    MBS_LIB_PATH, msg = fetch_github_repo(
        user = "zudanievil",
        repo = "MiceBrainMriSegmentation",
        branch = "paper",
        dst_dir = Path("/content") # google VM directory for user contents
    )
    rmdir_rec(Path("/content/sample_data")) # remove some toy datasets provided by google

else:
    MBS_LIB_PATH = "../"

In [5]:
# set path to inkscape executable

if RUNTIME_IS_ON_GOOGLE_COLAB:
    !apt install inkscape -y 2>&1 > /dev/null
    !inkscape --version
    INKSCAPE_EXE = "inkscape"
else:
    INKSCAPE_EXE = ...  # if you run this notebook locally, you need to paste your shell command for inkscape here 



Extracting templates from packages: 100%
Inkscape 0.92.3 (2405546, 2018-03-11)


## 2a. creating folders ##
classes `IC.ImageFolderInfo`, `IC.OntologyFolderInfo`, `IC.SegmentationResultsFolderInfo` contain `.write()` method that allows to conveniently create these folders in the filesystem

In [ ]:
working_dir = Path(WORKING_DIR)
temp_dir = (working_dir / "temp").resolve()


working_dir.mkdir(exist_ok=False)
temp_dir.mkdir(exist_ok=False)

In [4]:
sys.path.insert(0, MBS_LIB_PATH)
from mbs_lib.core import info_classes as IC


infos = {}

infos["ontology"] = IC.OntologyFolderInfo(working_dir/"ontology")
infos["ontology"].write()

for i in {1, 2}:
    
    infos[f"images_{i}"] = IC.ImageFolderInfo(working_dir / f"images_{i}")
    infos[f"images_{i}"].write()

    infos[f"results_{i}"] = IC.SegmentationResultFolderInfo(
        infos[f"images_{i}"], infos["ontology"], folder=working_dir / f"results_{i}")
    infos[f"results_{i}"].write()

## 2b. putting files into image folder ##
* `lib/default_specifications` folder has configuration files that are copied by the `.write()` methods of info classes into the directory that they create. These are YAML files, that can be used to correct the `lib.pipelines` modules' behaviour.
* `image_folder_specification.yml` that was is created in `c:/users/user/example/img` must be edited.
* under the `file_name_fields` key there is a list of strings, that compose any file name that is related to image. one should give these fields names that correspond to current experiment parameters, like time, mice strain, etc.
    > e.g. if configuration looks like this:
    file_name_fields:
        - a
        - b
        - c
    then image file names are assumed to look like `{a}_{b}_{c}.npy`, and metadata names like `{a}_{b}_{c}.yml`. these names will be broken down by some scripts into "dictionaries" of {field_name: field_value}. For instance, name 10_20_xyz.npz will correspond to {"a":"10", "b":"20", "c":"xyz"}.  **It is possible to adjust cropped scan resolution based on the value of "frame" field through editing `cropped_image_shapes` in the image folder configuration.**
* After that transfer the images into the `img_raw` subfolder of image folder, rename them according to the fields.

**For demonstration purposes a small portion of our data is retrieved and visualized below**

In [6]:
fetch_unpack_zip("https://drive.google.com/uc?export=download&id=1sqmcICO01UdB6WUxvYcVsJVNlnvAVKBS", temp_dir)
example_data = temp_dir / "example"

merge_folders(example_data, working_dir, "replace")
rmdir_rec(example_data)

## 3. manual scan segmentation ##
* manual scan segmentation is done by ImageJ program.
* the result of the segmentation for each image is couple files that contain digits separated by tabulations and newlines (ImageJ scripts do not have sophisticated tools for file parsing and formatting). These files are collected by a python script into a single YAML metadata file, based on the values under `metadata keys` in the image folder configuration.
* to segment images do the following:
    1. make list of file names in `img_raw` subfolder, put it in the `ij_img_list.txt` file in the image folder, separated by newlines (only names, not the full paths). Create `ij_pointer.txt`, put 0 into it. Here is an easy way to o this:

In [7]:
def list_raw_images(ifi: IC.ImageFolderInfo):
    img_list_fname = ifi.folder()/"ij_img_list.txt"
    img_idx_fname = ifi.folder()/"ij_pointer.txt"

    with open(img_list_fname, "wt") as f:
        for img in ifi.raw_iter():
            print(img.name(), file=f)
            
    with open(img_idx_fname, "wt") as f:
        print(0, file=f)


for i in {1, 2}:
    list_raw_images(infos[f"images_{i}"])

**Manual segmentation results in `.txt` files, which were fetched earlier along with images.**

*   
   2. Open the `lib/imagej_scripts/brain_segmentation_with_imagej.txt`, edit command for image opening according to [ImageJ macro guide] and [ImageJ macro functions list] to open your images (the function below comment `//IMAGE OPENING FUNCTION`). Initially it opens 32bit little-endian single channel 256x256, written as a C-array.
   1. Open imagej, load that file as imagej macro: `macros > install > imagej`
   1. Press `h` to open log window with the instructuions. follow the instructions.
   1. Run collect `collect_image_metadata.main()` (cell below)
   >you can find about metadata collection process and how to tune it with configuration editing if you run `help(collect_image_metadata.main)`
   
[ImageJ macro guide]: https://imagej.nih.gov/ij/developer/macro/macros.html#tools
[ImageJ macro functions list]: https://imagej.nih.gov/ij/developer/macro/functions.html

In [8]:
from mbs_lib.pipelines import collect_image_metadata
for i in {1, 2}:
    collect_image_metadata.main(infos[f"images_{i}"])

## 4a. downloading atlas images ##
variables for downloading are stored in `ontology_folder_specification.yml` file inside ontology folder that you've created above.

1. Edit `atlas_id` and `svg_groups` (a table of them can be found [here](http://help.brain-map.org/display/api/Atlas+Drawings+and+Ontologies))

In [9]:
from mbs_lib.pipelines import download_svgs
download_svgs.download_default_ontology(infos["ontology"])

default ontology written to /content/example/ontology/onts/default.xml


2. Edit the `slice_coordinates` if you know the coordinates for the first slice and the last slice in the atlas. this allows you to download a list of slice ids and zip it with coordinate range using a script in the cell below (the table is just for utility purposes, you can skip this step).

In [10]:
download_svgs.download_slice_ids_table(infos["ontology"])

saved section ids to /content/example/ontology/slice_ids.txt


Now you have a very readable (for an xml) `default.xml` tree of structures for the atlas, and probably a tabulation-separated table of slice ids.
For each frame value in image names there should be an svg image with the same name. fill the ontology folder configuration  `svg_names_and_slice_ids` with pairs of `frame_field_value: slice_id`. run the script from the cell below to bulk-download the atlas vector images.

In [11]:
# we've renamed the files in ontology folder config 
# so that they wouldn't overwrite svgs downloaded with example
# feel free to explore the differences
download_svgs.download_svgs(infos["ontology"])

downloaded: /content/example/ontology/svgs/f14_downloaded.svg
downloaded: /content/example/ontology/svgs/f17_downloaded.svg


You can and need to manually edit the `.svg` masks with Inkscape, but note that some Inkscape functions can alter all xml attributes of the paths. structures for rendering are found by `structure_id` attributes of the paths, make sure, they are not deleted in the editing process.
1. I advise to compute mean of all images for each frame and put it into a corresponding svg (the rendering pipeline ignores visible objects that do not have `structure_id` attribute, so you can leave image there).
2. You **must** create a rectangle with xml attribute `structure_id='bbox'`, that will mark the border of the rendered masks (to edit xml attributes of objects from Inkscape, open the 'xml editor' tab.
3. It's a good idea to slightly adjust nodes of atlas structures to fit the contours of the brain.
4. Note that you can specify an arbitrary function to modify each mask right before the scan segmentation. Therefore you do need to flip/copy/crop the svg masks if only one hemisphere is covered and you need masks for both hemisperes, or otherwise. 

## 4b. rendering raster masks ##
For this one needs to edit `inkscape_executable_path` value in the ontology folder's configuration. 
The rendering pipeline uses image folder configuration `cropped_image_shapes` values for determining mask sizes.
Details about process can be found in `prerender_masks.main` function documentation. This process might take several hours, but is easily parallelized by calling pipeline from different processes for only a part of all frames.

In [12]:
# let's set up inkscape executable command:
file_backup(infos["ontology"].configuration_path())
modify_info_folder_config(
    info = infos["ontology"], 
    keys = ["rendering_constants", "inkscape_executable"], 
    value = INKSCAPE_EXE)

In [13]:
from mbs_lib.pipelines import prerender_masks
# prerender_masks needs `ImageFolderInfo` configuration file to acqire shapes for masks.
# we will use masks prerendered for image folder 1 for both folder 1 and folder 2. 
prerender_masks.main(infos["ontology"], infos["images_1"], frames = ["f14", "f17"]) # we specify frames so that only these svgs will be used for mask generation

rendering for f14: 100%|█████████▉| 1326/1327 [00:24<00:00, 112.42it/s, Retina]               
Total found structures 78
rendering for f17: 100%|█████████▉| 1326/1327 [00:07<00:00, 144.27it/s, Retina]               
Total found structures 15


## 5. preprocessing scans ##
* Convert images to numpy native array format via `numpy.save()`, put them into `img` subfolder of the image folder.

* The pipeline called below uses `cropped_image_shapes` key of the configuration and `frame` field value in the image name as well as some metadata keys to crop and resize the images.

In [14]:
import numpy as np


def convert_raw_to_npy(ifi: IC.ImageFolderInfo, dtype="<i4", shape=(256, 256)):
    for img_info in ifi.raw_iter():
        img = img_info.raw_image(dtype=dtype, shape=shape)
        np.save(img_info.image_path(), img)


for i in {1, 2}:
    convert_raw_to_npy(infos[f"images_{i}"])

In [15]:
from mbs_lib.pipelines import crop_images

for i in {1, 2}:
    crop_images.main(infos[f"images_{i}"])

To normalize image by intensity, we generate reference values and put them into metadata files. Later on you can choose which key to to use for image normalization. 

In [16]:
from mbs_lib.pipelines import generate_intensity_reference

for i in {1, 2}:
    generate_intensity_reference.main(infos[f"results_{i}"])

## 6. fine anatomical segmentation
1. You need to specify a tabulation-separated table which contains names of images. Table header specifies, from which columns images fall into 'control' group and from which -- into the 'effect' group. images from the one row constitute a batch and are compared against each other. the `batch_for_comparison` pipeline can create this table. Note that one can easily edit this table in MS Excel/Libre Office, etc.
You can find about how to control pipeline behaviour through a result folder configuration in the `batch_for_comparison.main` description.

In [17]:
# change reference hour and title for debugging plots

ref_hour = "3"
plot_title_fstring = "3h vs {hour}, frame: {frame} p-value and mean relative intensity"

for i in {1, 2}:
    file_backup(infos[f"results_{i}"].configuration_path())
    modify_info_folder_config(
        info = infos[f"results_{i}"], 
        keys = ["batching", "comparison_reference_value"], 
        value = ref_hour,
    )
    modify_info_folder_config(
        info = infos[f"results_{i}"],
        keys = ["comparison", "plot_title"],
        value = plot_title_fstring,
    )

In [18]:
from mbs_lib.pipelines import batch_for_comparison

for i in {1, 2}:
    batch_for_comparison.main(infos[f"results_{i}"])

2. please read the `compare_and_segment.main` help for detalis on how to control it's behaviour with configuration file and some other files, that can be created in the `spec` subfolder of results folder.

In [19]:
from mbs_lib.pipelines import compare_and_segment
help(compare_and_segment.main)

Help on function main in module mbs_lib.pipelines.compare_and_segment:

main(segmentation_result_folder_info: Union[ForwardRef('SegmentationResultFolderInfo'), pathlib.Path, str], batch_range: slice = None, save_intersection_images: bool = True)
    for each batch of animals (see `pipelines.batch_for_comparison`)
    1. compare images with specified statistic (see results folder configuration, `comparison/image_comparison_type`)
    2. plot p-values (false positive rate), difference between images (if `save_intersection_images=True`)
    3. for differen p-value levels (see configuration `comparison/pval_thesholds`),
    calculate number of significant pixels, mean and standard deviation.
    calculated statistics are stored in "pickled" DataFrames,
    they need to be aggregated by `pipelines.compare_and_segment.collect_segmentation_results` function
    
    NB: one can specify a set of anatomical structures to use in the file
    at location given by `segmentation_results_folder_info

In [20]:
from mbs_lib.pipelines import compare_and_segment

for i in {1, 2}:
    print(f"processing: results_{i}")
    compare_and_segment.main(infos[f"results_{i}"])

processing: results_1


  0%|          | 0/2 [00:00<?, ?it/s, 0]/content/zudanievil-MiceBrainMriSegmentation/mbs_lib/pipelines/compare_and_segment.py:185: UserWarning: No contour levels were found within the data range.
  axs[0].contour(structure_mask, **spec['contour_kwargs'])
/content/zudanievil-MiceBrainMriSegmentation/mbs_lib/pipelines/compare_and_segment.py:186: UserWarning: No contour levels were found within the data range.
  axs[1].contour(structure_mask, **spec['contour_kwargs'])


processing: results_2


3. segmentation results are saved into a temporary folder to save RAM and allow parallel execution (by running several processes and supplying `batch_range` argument to  `compare_and_segment.main`). Because of this design choice, segmentation results' pieces are aggregated by a separate function. You also need to delete temp folder yourself.

In [21]:
# %%debug -b /content/zudanievil-MiceBrainMriSegmentation-c417baf/mbs_lib/pipelines/compare_and_segment.py:307
for i in {1, 2}:
    compare_and_segment.collect_segmentation_results(infos[f"results_{i}"])
    # rmdir_rec(infos[f"results_{i}"].segmentation_temp())

## 7. aftermath
There are some scripts for processing the results table. Consult with their descriptions to understand what do they do.
also there is an execute_all script, that just calls them in a logical order (some scripts depend on each other). They also use configuration for the results folder.

In [22]:
structures = (
    'Main Olfactory Bulb',
    'Olfactory Nerve',
    'Anterior Olfactory Nucleus',
    'Piriform Area',
    'Piriform-Amygdalar Area',
    'Cortical Amygdalar Area',
    'Entorhinal Area',
    'Olfactory Tubercle',
)

In [27]:
from mbs_lib.pipelines import segmentation_results_processing

for i in {1, 2}:
    segmentation_results_processing.execute_all(infos[f"results_{i}"])

/content/zudanievil-MiceBrainMriSegmentation/mbs_lib/pipelines/segmentation_results_processing.py:66: UserWarning: make_kinetics_table subpipeline have not been generalized for arbitrary filename fields, please inspect code
  warnings.warn(message='make_kinetics_table subpipeline have not been generalized '


KeyError: ignored